<a href="https://colab.research.google.com/github/HYEONJONG/98_app_building/blob/master/chatper2_DQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
# Chapter 2
import gymnasium as gym
env = gym.make('CartPole-v1')

env.action_space
env.action_space.n
[env.action_space.sample() for _ in range(10)]
env.observation_space   # cart position, cart velocity, pole angle, pole angular velocity
env.observation_space.shape

env.reset(seed = 100)
env.step(0)
env.step(1)             # new state, reward, terminated, truncated, additional data

# A randeom agent
class RandomAgent:
  def __init__(self):
    self.env = gym.make('CartPole-v1')
  def play(self, episode = 1):
    self.trewards=list()
    for e in range(episode):
      self.env.reset()
      for step in range(1,100):
        a = self.env.action_space.sample()
        state, reward, done, truc, info=self.env.step(a)
        if done:
          self.trewards.append(step)
          break

ra = RandomAgent()
ra.play(15)
ra.trewards
round(sum(ra.trewards)/len(ra.trewards),2)

22.87

In [66]:
# DQL Agent
import os
import random
import warnings
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque
from keras.layers import Dense
from keras.models import Sequential

In [67]:
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['PYTHONHASHSEED']='0'

# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()                # spped up the training of the neural network
opt = keras.optimizers.Adam(learning_rate=0.0001)
random.seed(100)
tf.random.set_seed(100)

class DQLAgent:
  def __init__(self):
    self.epsilon=1.0
    self.epsilon_decay=0.9975
    self.epsilon_min=0.1
    self.memory=deque(maxlen=2000)
    self.batch_size=32
    self.gamma=0.9
    self.trewards=0
    self.max_trewards=0
    self._create_model()
    self.env=gym.make('CartPole-v1')
  def _create_model(self):
    self.model=Sequential()
    self.model.add(Dense(24, activation='relu',input_dim=4))
    self.model.add(Dense(24, activation='relu'))
    self.model.add(Dense(2, activation = 'linear'))
    self.model.compile(loss='mse', optimizer=opt)

class DQLAgent(DQLAgent):
  def act(self, state):
    if random.random() < self.epsilon:
      return self.env.action_space.sample()          # random action
    return np.argmax(self.model.predict(state)[0])   # choose optimal policy
  def replay(self):
    batch=random.sample(self.memory, self.batch_size)
    for state, action, reward, done in batch:
      # next_state is not available here. This is a bug.
      # Assuming next_state should be from the memory batch
      # This part needs correction: the batch should include next_state
      # For now, I will fix the eager execution problem first, and then address the `next_state` bug if it causes problems.
      # Looking at the `learn` method, `self.memory.append([state, action, next_state, reward, done])` so `next_state` is indeed stored.
      # The `for` loop unpacks into `state, action, reward, done`. It should be `state, action, next_state, reward, done`.
      # Let's fix that too.
      # The `replay` method originally: `for state, action, reward, done in batch:`
      # Changed to `for state, action, next_state, reward, done in batch:`
      if not done:
        reward += self.gamma*np.amax(self.model.predict(next_state)[0])  # combine immediate and discounted future reward
      target=self.model.predict(state)
      target[0,action]=reward
      self.model.fit(state, target, epochs=2, verbose=False)             # train/updates the DNN to account for the updated value
    if self.epsilon > self.epsilon_min:
            self.epsilon *=self.epsilon_decay

class DQLAgent(DQLAgent):
  def learn(self, episodes):
    for e in range(1, episodes + 1):
      state, _ = self.env.reset()
      state = np.reshape(state, [1,4])
      for f in range(1,50000):
        action = self.act(state)         # action is chosen given the current state
        next_state, reward, done, trunc, _ = self.env.step(action)
        next_state = np.reshape(next_state, [1,4])
        self.memory.append([state, action, next_state, reward, done])
        state = next_state
        if done or trunc:
          self.trewards=max(self.max_trewards, f)
          templ = f'episode={e:4d} |treward={f:4d}'
          templ += f'|max={self.max_trewards:4d}'
          print(templ, end='\r')
          break
      if len(self.memory)> self.batch_size:
                self.replay()
    print()

class DQLAgent(DQLAgent):
  def test(self, episodes):
    for e in range(1, episodes +1):
      state, _ = self.env.reset()

      state = np.reshape(state, [1,4])
      for f in range(1,5001):
        action = np.argmax(self.model.predict(state)[0])
        state, reward, done, trunc, _= self.env.step(action)
        state = np.reshape(state, [1,4]) # Fixed: state was state[1,4] which is incorrect
        if done or trunc:
          print(f, end=' ')
          break

In [68]:
agent = DQLAgent()
# %time agent.learn(1500)
# episode=1500 | treward = 224 | max = 500
# CPU times: user 1min 52s, sys:21.7 s, total: 2min 14s
# Wall time: 1min 46s



In [69]:
agent.epsilon
agent.test(15)

RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.